# Milligram Analysis of Salt Lake City, UT Claims '17-'18

For Valisure Metformin & Oncologic Study

Last updated: July 22, 2020

### Preliminary Tasks

In [1]:
# Import useful libraries
import os
os.chdir('/Users/michaelbecker/Desktop/Milligram/Valisure Metformin Publication/Data & Analysis')
import pandas as pd
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 50)
pd.options.mode.chained_assignment = None
import numpy as np
import re
import matplotlib.pyplot as plt
import codecs
import scipy.stats as stats

# Load in Relevant Datasets
slc_claims = pd.read_csv('Milligram_Rx_Extract_2017_2018_SLConly.txt', sep = '|')
valisure_met_analysis = pd.read_excel('(Confidential) Valisure Metformin Analysis Results Overview 070920.xlsx', sheet_name = 0)
valisure_met_analysis_tested = pd.read_excel('(Confidential) Valisure Metformin Analysis Results Overview 070920.xlsx', sheet_name = 2)
milligram_met = pd.read_excel('Metformin - All of Utah - 2017.xlsx', usecols = 'F:K', )
oncologics = pd.read_excel('Oncologic Drug Classification 072020.xlsx')
ndcs = pd.read_csv('NDC Database 071420.txt', sep = '\t', lineterminator = '\n', encoding = 'unicode_escape')
ndcs_unapproved = pd.read_excel('NDC Database Unapproved 071520.xlsx')

### Clean Data

##### Valisure Data

In [2]:
# Select relevant columns to tested Metformin
valisure_met_analysis_tested = valisure_met_analysis_tested.fillna(0)
valisure_met_analysis_tested_s = valisure_met_analysis_tested.iloc[:, [0, 1, 7]]

# Create 9-digit NDCs for tested Metformin
valisure_met_tested_ndcs = list(valisure_met_analysis_tested_s['NDC Product Code'])
for drug in range(len(valisure_met_tested_ndcs)):
    if re.search(r'^[0-9]{4}\-[0-9]{4}$', valisure_met_tested_ndcs[drug]):
        valisure_met_tested_ndcs[drug] = '0' + valisure_met_tested_ndcs[drug]
    elif re.search(r'^[0-9]{5}\-[0-9]{3}$', valisure_met_tested_ndcs[drug]):
        valisure_met_tested_ndcs[drug] = valisure_met_tested_ndcs[drug][:6] + '0' + valisure_met_tested_ndcs[drug][6:]
    valisure_met_tested_ndcs[drug] = re.sub(r'[^0-9]', '', valisure_met_tested_ndcs[drug])
valisure_met_analysis_tested_s['NDC Product Code 9'] = valisure_met_tested_ndcs
valisure_met_analysis_tested_s['NDC Product Code 9'] = valisure_met_analysis_tested_s['NDC Product Code 9'].astype(str)

# Clean all Metformin
valisure_met_analysis = valisure_met_analysis.iloc[6:, :].reset_index(drop = True)
valisure_met_analysis['NDCs (Package Codes)'].iloc[13] = valisure_met_analysis['NDCs (Package Codes)'].iloc[13].replace(u'\xa0', ' ')
valisure_met_analysis_s = valisure_met_analysis.iloc[:, [0, 10, 14]]
valisure_met_analysis_s = valisure_met_analysis_s.assign(NDC_Package_Codes = valisure_met_analysis_s['NDCs (Package Codes)'].str.split(', ')).explode('NDC_Package_Codes')

# Create 9-digit NDCs for all Metformin
valisure_met_ndcs = list(valisure_met_analysis_s['NDC_Package_Codes'])
for drug in range(len(valisure_met_ndcs)):
    if re.search(r'^[0-9]{4}\-[0-9]{4}\-[0-9]{2}$', valisure_met_ndcs[drug]):
        valisure_met_ndcs[drug] = '0' + valisure_met_ndcs[drug]
    elif re.search(r'^[0-9]{5}\-[0-9]{3}\-[0-9]{2}$', valisure_met_ndcs[drug]):
        valisure_met_ndcs[drug] = valisure_met_ndcs[drug][:6] + '0' + valisure_met_ndcs[drug][6:]
    valisure_met_ndcs[drug] = re.sub(r'[^0-9]', '', valisure_met_ndcs[drug])
valisure_met_analysis_s['NDC Package Code 11'] = valisure_met_ndcs
valisure_met_analysis_s['NDC Package Code 11'] = valisure_met_analysis_s['NDC Package Code 11'].astype(str)
valisure_met_analysis_s['NDC Package Code 9'] = valisure_met_analysis_s['NDC Package Code 11'].str[:9]

# Create full Valisure lookup table
valisure_met_ratings = pd.merge(valisure_met_analysis_s, valisure_met_analysis_tested_s, left_on = 'NDC Package Code 9', right_on = 'NDC Product Code 9', how = 'left').iloc[:, [5, 2, 8]]
valisure_met_ratings.columns = ['NDC', 'Rating_Assoc', 'Rating_Test']
valisure_met_ratings = valisure_met_ratings.drop_duplicates().reset_index(drop = True)
valisure_met_ratings = valisure_met_ratings.fillna('unknown')

##### Milligram Data

In [3]:
# Clean Milligram Metformin
milligram_met_ndcs = milligram_met.iloc[:, [0]].dropna().astype(str)
milligram_met_ndcs['NDC'] = milligram_met_ndcs['NDC'].str.zfill(13)
milligram_met_ndcs = list(milligram_met_ndcs['NDC'].str[:11])

##### All Metformin Data

In [4]:
# Create full lists of Package and Product NDCs for Metformin
met_package_ndcs = list(set(valisure_met_ndcs)|set(milligram_met_ndcs))
met_product_ndcs = [sl[:9] for sl in met_package_ndcs]
met_product_ndcs = set(met_product_ndcs)
met_product_ndcs_notest = met_product_ndcs - set(valisure_met_tested_ndcs)

##### NDC Data

In [5]:
# Clean NDC data for joins
ndcs.iloc[:, -1] = ndcs.iloc[:, -1].str.rstrip()
ndcs = ndcs.rename(columns = { 'LISTING_RECORD_CERTIFIED_THROUGH\r':'LISTING_RECORD_CERTIFIED_THROUGH' })
ndcs_cond = ndcs.iloc[:, [3, 5, 1]]
ndcs_cond['PROPRIETARYNAME'] = ndcs_cond['PROPRIETARYNAME'].str.lower()
ndcs_cond['NONPROPRIETARYNAME'] = ndcs_cond['NONPROPRIETARYNAME'].str.lower()
ndcs_unapproved_cond = ndcs_unapproved.iloc[:, [3, 9, 1]]
ndcs_unapproved_cond['NONPROPRIETARYNAME'] = ndcs_unapproved_cond['NONPROPRIETARYNAME'].str.lower()
ndcs_unapproved_cond['SUBSTANCENAME'] = ndcs_unapproved_cond['SUBSTANCENAME'].str.lower()

##### Oncologic Data

In [6]:
# Join Oncologics with NDCs
oncologics_ndcs = pd.merge(oncologics, ndcs_cond, left_on = 'drugname', right_on = 'NONPROPRIETARYNAME', how = 'left')
oncologics_ndcs_F = pd.merge(oncologics_ndcs[oncologics_ndcs['NONPROPRIETARYNAME'].isna()], ndcs_cond, left_on = 'drugname', right_on = 'PROPRIETARYNAME', how = 'left')
### oncologics_ndcs_F.dropna().count()
oncologics_ndcs_F = oncologics_ndcs_F.iloc[:, :2]
oncologics_ndcs_T = oncologics_ndcs.dropna()
oncologics_ndcs_T['NDC_Status'] = 'approved'

# Join Oncologics with Unapproved NDCs
oncologics_ndcs_un = pd.merge(oncologics_ndcs_F, ndcs_unapproved_cond, left_on = 'drugname', right_on = 'NONPROPRIETARYNAME', how = 'left')
oncologics_ndcs_un_F = pd.merge(oncologics_ndcs_un[oncologics_ndcs_un['NONPROPRIETARYNAME'].isna()], ndcs_unapproved_cond, left_on = 'drugname', right_on = 'SUBSTANCENAME', how = 'left')
### oncologics_ndcs_un_F.dropna().count()
oncologics_ndcs_un_T = oncologics_ndcs_un.dropna()
oncologics_ndcs_un_T['NDC_Status'] = 'unapproved'

# Create full Oncologic NDC table
oncologics_ndcs_full = oncologics_ndcs_T.append(oncologics_ndcs_un_T).iloc[:, [0, 1, 6, 2, 3, 4, 5]]

# Re-format NDCs for existing Oncologic codes
oncologics_ndcs_only = oncologics_ndcs_full['PRODUCTNDC'].reset_index(drop = True)
for drug in range(len(oncologics_ndcs_only)):
    if re.search(r'^[0-9]{4}\-[0-9]{4}$', oncologics_ndcs_only[drug]):
        oncologics_ndcs_only[drug] = '0' + oncologics_ndcs_only[drug]
    elif re.search(r'^[0-9]{5}\-[0-9]{3}$', oncologics_ndcs_only[drug]):
        oncologics_ndcs_only[drug] = oncologics_ndcs_only[drug][:6] + '0' + oncologics_ndcs_only[drug][6:]
    oncologics_ndcs_only[drug] = re.sub(r'[^0-9]', '', oncologics_ndcs_only[drug])
oncologics_ndcs_full['PRODUCTNDC9'] = oncologics_ndcs_only
oncologics_ndcs_full['PRODUCTNDC9'] = oncologics_ndcs_full['PRODUCTNDC9'].astype(str)

##### Claims Data

In [7]:
# Create 9-digit NDC for Claims
slc_claims['RX_NDC_CODE_9'] = slc_claims['RX_NDC_CODE'].str[:9]

### Join Metformin and Oncologic Tables

In [8]:
# Find overlapping MEMBER_ID values for Valisure tested Metformin
met_claims_tested = slc_claims[slc_claims['RX_NDC_CODE_9'].isin(valisure_met_tested_ndcs)]
met_claims_tested_ids = list(met_claims_tested['MEMBER_KEY'].unique())
oncologic_claims = slc_claims[slc_claims['RX_NDC_CODE_9'].isin(oncologics_ndcs_only)]
oncologic_claims_ids = list(oncologic_claims['MEMBER_KEY'].unique())
overlap_tested_ids = list(set(met_claims_tested_ids) & set(oncologic_claims_ids))

# Find overlapping MEMBER_ID values for all Metformin
met_claims = slc_claims[slc_claims['RX_NDC_CODE_9'].isin(met_product_ndcs)]
met_claims_ids = list(met_claims['MEMBER_KEY'].unique())
overlap_ids = list(set(met_claims_ids) & set(oncologic_claims_ids))

# Find overlapping MEMBER_ID values for non-tested Metformin
met_claims_notest = slc_claims[slc_claims['RX_NDC_CODE_9'].isin(met_product_ndcs_notest)]
met_claims_notest_ids = list(met_claims['MEMBER_KEY'].unique())
overlap_notest_ids = list(set(met_claims_notest_ids) & set(oncologic_claims_ids))

# Create overlap dataframes
overlap_all = slc_claims[slc_claims['MEMBER_KEY'].isin(overlap_ids)]
overlap_all = pd.merge(overlap_all, valisure_met_ratings, left_on = 'RX_NDC_CODE_9', right_on = 'NDC', how = 'left')
overlap_tested = slc_claims[slc_claims['MEMBER_KEY'].isin(overlap_tested_ids)]
overlap_tested = pd.merge(overlap_tested, valisure_met_ratings, left_on = 'RX_NDC_CODE_9', right_on = 'NDC', how = 'left')
overlap_notest = slc_claims[slc_claims['MEMBER_KEY'].isin(overlap_notest_ids)]

### Information on Overlap (Metformin + Oncologics) Data

This section examines the number of individuals that filed claims for Metformin (tested by Valisure) and Oncologics.

In [9]:
# Assoc # Number of unique individuals taking any Metformin and cancer drugs
print(len(overlap_ids), 'unique Member IDs for NDCs associated with Assoc Metformin')
print()

# Assoc # Above ages relative to 65
overlap_all_ind = overlap_all.iloc[:, :3].drop_duplicates().sort_values(by = 'MEMBER_KEY').reset_index()
print(overlap_all_ind[overlap_all_ind['MEM_AGE_GROUP_EOY'] > '65'].count()[2], 'are 65+')
print(overlap_all_ind[overlap_all_ind['MEM_AGE_GROUP_EOY'] < '65'].count()[2], 'are younger')
print()

# Test # Number of unique individuals taking tested Metformin and cancer drugs
print(len(overlap_tested_ids), 'unique Member IDs for NDCs of Tested Metformin')
print()

# Test # Above ages relative to 65
overlap_tested_ind = overlap_tested.iloc[:, :3].drop_duplicates().sort_values(by = 'MEMBER_KEY').reset_index()
print(overlap_tested_ind[overlap_tested_ind['MEM_AGE_GROUP_EOY'] > '65'].count()[2], 'are 65+')
print(overlap_tested_ind[overlap_tested_ind['MEM_AGE_GROUP_EOY'] < '65'].count()[2], 'are younger')

221 unique Member IDs for NDCs associated with Assoc Metformin

124 are 65+
122 are younger

197 unique Member IDs for NDCs of Tested Metformin

106 are 65+
112 are younger


Given the Metformin testing data that Valisure have provided, there are two types of NDCs:

* First, those NDCs of Metformin that were explicitly tested (Test) (Sheet 2 of Valisure's file)
* Second, the NDCs tested above with the NDCs associated/from the same producer but not tested themselves (Assoc) (Sheet 1 of Valisure's file)

I designate the latter group as 'Associated' or 'Assoc', which carry the rating given to the NDCs they're associated with, and the former group as 'Test', which were themselves explicitly tested. 

This means that there are two potential samples -- the 221 that took an associated NDC or the 197 that took an explicitly tested NDC. So, depending on desired methodology, I've included two iterations of all the following information:

* First, a sample of those that took NDCs of Associated Metformin with all associated ratings for those NDCs
* Second, a sample of those that took NDCs of Tested Metformin with the associated ratings for those NDCs

Each block is indicated by either **Test** or **Assoc** for the corresponding information.

In [10]:
# Assoc # All Overlappers taking Red/Yellow/Green Associated doses of Metformin
overlap_all_ratings_assoc = overlap_all.groupby(['MEMBER_KEY', 'Rating_Assoc'])[['MEMBER_KEY', 'Rating_Assoc']].size().reset_index()
overlap_all_ratings_assoc = overlap_all_ratings_assoc.rename(columns = { 0:'#_Prescriptions' })
overlap_all_ratings_assoc['Tainted_Test'] = 'NA'
for member in range(len(overlap_ids)):
    ind = list(overlap_all_ratings_assoc[overlap_all_ratings_assoc['MEMBER_KEY'] == overlap_ids[member]]['Rating_Assoc'].index)
    if any(overlap_all_ratings_assoc[overlap_all_ratings_assoc['MEMBER_KEY'] == overlap_ids[member]]['Rating_Assoc'] == 'red'):
        overlap_all_ratings_assoc['Tainted_Test'].iloc[ind] = 2
    elif any((overlap_all_ratings_assoc[overlap_all_ratings_assoc['MEMBER_KEY'] == overlap_ids[member]]['Rating_Assoc'] == 'yellow')) and all(overlap_all_ratings_assoc[overlap_all_ratings_assoc['MEMBER_KEY'] == overlap_ids[member]]['Rating_Assoc'] != 'red'):
        overlap_all_ratings_assoc['Tainted_Test'].iloc[ind] = 1
    else:
        overlap_all_ratings_assoc['Tainted_Test'].iloc[ind] = 0
overlap_all_ratings_assoc = overlap_all_ratings_assoc.iloc[overlap_all_ratings_assoc.iloc[:, [0, 3]].drop_duplicates().index]
ids_tainted_red_assoc = overlap_all_ratings_assoc[overlap_all_ratings_assoc['Tainted_Test'] == 2].count()[0]
ids_tainted_yellow_assoc = overlap_all_ratings_assoc[overlap_all_ratings_assoc['Tainted_Test'] == 1].count()[0]
ids_clean_assoc = len(overlap_all_ratings_assoc) - ids_tainted_red_assoc - ids_tainted_yellow_assoc
print((ids_tainted_red_assoc + ids_tainted_yellow_assoc), 'out of 221 IDs took tainted (associated) Metformin')
print(ids_tainted_red_assoc, 'IDs took some Red rated Metformin')
print(ids_tainted_yellow_assoc, 'IDs took some Yellow rated Metformin (but no Red)')
print(ids_clean_assoc, 'IDs were clean or unknown')
print('Red rate =', round(ids_tainted_red_assoc/len(overlap_all_ratings_assoc), 3))
print('Yellow rate =', round(ids_tainted_yellow_assoc/len(overlap_all_ratings_assoc), 3))
print('Tainted (assoc) rate =', round((ids_tainted_red_assoc + ids_tainted_yellow_assoc)/len(overlap_all_ratings_assoc), 3))

154 out of 221 IDs took tainted (associated) Metformin
43 IDs took some Red rated Metformin
111 IDs took some Yellow rated Metformin (but no Red)
62 IDs were clean or unknown
Red rate = 0.199
Yellow rate = 0.514
Tainted (assoc) rate = 0.713


In [11]:
# Test # Tested Overlappers taking Red/Yellow/Green Tested doses of Metformin
overlap_tested_ratings_test = overlap_tested.groupby(['MEMBER_KEY', 'Rating_Test'])[['MEMBER_KEY', 'Rating_Test']].size().reset_index()
overlap_tested_ratings_test = overlap_tested_ratings_test.rename(columns = { 0:'#_Prescriptions' })
overlap_tested_ratings_test['Tainted_Test'] = 'NA'
for member in range(len(overlap_ids)):
    ind = list(overlap_tested_ratings_test[overlap_tested_ratings_test['MEMBER_KEY'] == overlap_ids[member]]['Rating_Test'].index)
    if any(overlap_tested_ratings_test[overlap_tested_ratings_test['MEMBER_KEY'] == overlap_ids[member]]['Rating_Test'] == 'red'):
        overlap_tested_ratings_test['Tainted_Test'].iloc[ind] = 2
    elif any(overlap_tested_ratings_test[overlap_tested_ratings_test['MEMBER_KEY'] == overlap_ids[member]]['Rating_Test'] == 'yellow') and all(overlap_tested_ratings_test[overlap_tested_ratings_test['MEMBER_KEY'] == overlap_ids[member]]['Rating_Test'] != 'red'):
        overlap_tested_ratings_test['Tainted_Test'].iloc[ind] = 1
    else:
        overlap_tested_ratings_test['Tainted_Test'].iloc[ind] = 0
overlap_tested_ratings_test = overlap_tested_ratings_test.iloc[overlap_tested_ratings_test.iloc[:, [0, 3]].drop_duplicates().index]
ids_tainted_red_test = overlap_tested_ratings_test[overlap_tested_ratings_test['Tainted_Test'] == 2].count()[0]
ids_tainted_yellow_test = overlap_tested_ratings_test[overlap_tested_ratings_test['Tainted_Test'] == 1].count()[0]
ids_clean_test = len(overlap_tested_ratings_test) - ids_tainted_red_test - ids_tainted_yellow_test
print((ids_tainted_red_test + ids_tainted_yellow_test), 'out of 197 IDs took tainted (test) Metformin')
print(ids_tainted_red_test, 'IDs took some Red rated Metformin')
print(ids_tainted_yellow_test, 'IDs took some Yellow rated Metformin (but no red)')
print(ids_clean_test, 'IDs were clean/unknown')
print('Red rate =', round(ids_tainted_red_test/len(overlap_tested_ratings_test), 3))
print('Yellow rate =', round(ids_tainted_yellow_test/len(overlap_tested_ratings_test), 3))
print('Tainted (test) rate =', round((ids_tainted_red_test + ids_tainted_yellow_test)/len(overlap_tested_ratings_test), 3))

143 out of 197 IDs took tainted (test) Metformin
34 IDs took some Red rated Metformin
109 IDs took some Yellow rated Metformin (but no red)
53 IDs were clean/unknown
Red rate = 0.173
Yellow rate = 0.556
Tainted (test) rate = 0.73


Below is information regarding the distribution of Metformin that overlap individuals in different groups took based upon Valisure ratings.

In [12]:
# Create overlap table and 'TOTAL_UNITS' for Tested Metformin
overlap_test = slc_claims.loc[slc_claims['MEMBER_KEY'].isin(overlap_tested_ids) & slc_claims['RX_NDC_CODE_9'].isin(met_product_ndcs)]
overlap_test = pd.merge(overlap_test, valisure_met_ratings, left_on = 'RX_NDC_CODE_9', right_on = 'NDC', how = 'left').drop('NDC', axis = 1)
for row in range(len(overlap_test)):
    if pd.isnull(overlap_test['Rating_Assoc'][row]):
        overlap_test['Rating_Assoc'][row] = 'unknown'
    if pd.isnull(overlap_test['Rating_Test'][row]):
        overlap_test['Rating_Test'][row] = 'unknown'
overlap_cond = overlap_test.iloc[:, [0, 1, 2, 5, 6, 8, 9, 10, 11, 12, 21, 22, 23]]
overlap_cond['TOTAL_UNITS'] = overlap_test['QTY_SVC_UNITS'] * overlap_test['RX_REFILLS']

In [13]:
# Assoc # Create 'TOTAL_UNITS' for proportions (Assoc)
overlap_props_assoc = overlap_cond.groupby(['MEMBER_KEY', 'Rating_Assoc'])['TOTAL_UNITS'].sum()
overlap_props_assoc_ind = overlap_cond.groupby(['MEMBER_KEY'])['TOTAL_UNITS'].sum()
overlap_props_assoc = overlap_props_assoc.div(overlap_props_assoc_ind, level = 'MEMBER_KEY').reset_index()
overlap_props_assoc['TOTAL_UNITS'] = overlap_props_assoc['TOTAL_UNITS'] * 100

# Create Table for proportion of each quality (Assoc)
overlap_props_assoc['Red_Prop'] = 0.0
overlap_props_assoc['Yellow_Prop'] = 0.0
overlap_props_assoc['Green_Prop'] = 0.0
overlap_props_assoc['Unknown_Prop'] = 0.0
for row in range(len(overlap_props_assoc)):
    if overlap_props_assoc['Rating_Assoc'][row] == 'red':
        overlap_props_assoc['Red_Prop'][row] = overlap_props_assoc['TOTAL_UNITS'][row]
    elif overlap_props_assoc['Rating_Assoc'][row] == 'yellow':
        overlap_props_assoc['Yellow_Prop'][row] = overlap_props_assoc['TOTAL_UNITS'][row]
    elif overlap_props_assoc['Rating_Assoc'][row] == 'green':
        overlap_props_assoc['Green_Prop'][row] = overlap_props_assoc['TOTAL_UNITS'][row]
    overlap_props_assoc['Unknown_Prop'][row] = 100 - (overlap_props_assoc['Red_Prop'][row] + overlap_props_assoc['Yellow_Prop'][row] + overlap_props_assoc['Green_Prop'][row])
overlap_props_assoc = overlap_props_assoc.drop(['Rating_Assoc'], axis = 1).groupby('MEMBER_KEY').sum()

# Fix proportions > 100% (Assoc)
for row in range(len(overlap_props_assoc)):
    if overlap_props_assoc['Unknown_Prop'].iloc[row] >= 200.0:
        overlap_props_assoc['Unknown_Prop'].iloc[row] -= 200
    if overlap_props_assoc['Unknown_Prop'].iloc[row] >= 100.0:
        overlap_props_assoc['Unknown_Prop'].iloc[row] -= 100
overlap_props_assoc = round(overlap_props_assoc, 3)

# Overlap proportions for Assoc group
overlap_props_assoc.mean()[1:]

# Information on Red and Yellow for Overlap Assoc
print('Proportion of individuals that took > 50% Red =', len(overlap_props_assoc[overlap_props_assoc['Red_Prop'] >= 50.0]) / len(overlap_props_assoc))
print('Proportion of individuals that took > 50% Red or Yellow =', len(overlap_props_assoc[(overlap_props_assoc['Red_Prop'] + overlap_props_assoc['Yellow_Prop']) >= 0.50]) / len(overlap_props_assoc))
print('Proportion of individuals that took 100% Red or Yellow =', len(overlap_props_assoc[(overlap_props_assoc['Red_Prop'] + overlap_props_assoc['Yellow_Prop']) == 100]) / len(overlap_props_assoc))

Proportion of individuals that took > 50% Red = 0.1065989847715736
Proportion of individuals that took > 50% Red or Yellow = 0.7360406091370558
Proportion of individuals that took 100% Red or Yellow = 0.4619289340101523


In [14]:
# Test # Create 'TOTAL_UNITS' for proportions (Test)
overlap_props_test = overlap_cond.groupby(['MEMBER_KEY', 'Rating_Test'])['TOTAL_UNITS'].sum()
overlap_props_test_ind = overlap_cond.groupby(['MEMBER_KEY'])['TOTAL_UNITS'].sum()
overlap_props_test = overlap_props_test.div(overlap_props_test_ind, level = 'MEMBER_KEY').reset_index()
overlap_props_test['TOTAL_UNITS'] = overlap_props_test['TOTAL_UNITS'] * 100

# Create Table for proportion of each quality (Test)
overlap_props_test['Red_Prop'] = 0.0
overlap_props_test['Yellow_Prop'] = 0.0
overlap_props_test['Green_Prop'] = 0.0
overlap_props_test['Unknown_Prop'] = 0.0
for row in range(len(overlap_props_test)):
    if overlap_props_test['Rating_Test'][row] == 'red':
        overlap_props_test['Red_Prop'][row] = overlap_props_test['TOTAL_UNITS'][row]
    elif overlap_props_test['Rating_Test'][row] == 'yellow':
        overlap_props_test['Yellow_Prop'][row] = overlap_props_test['TOTAL_UNITS'][row]
    elif overlap_props_test['Rating_Test'][row] == 'green':
        overlap_props_test['Green_Prop'][row] = overlap_props_test['TOTAL_UNITS'][row]
    overlap_props_test['Unknown_Prop'][row] = 100 - (overlap_props_test['Red_Prop'][row] + overlap_props_test['Yellow_Prop'][row] + overlap_props_test['Green_Prop'][row])
overlap_props_test = overlap_props_test.drop(['Rating_Test'], axis = 1).groupby('MEMBER_KEY').sum()

# Fix proportions > 100% (Test)
for row in range(len(overlap_props_test)):
    if overlap_props_test['Unknown_Prop'].iloc[row] >= 200.0:
        overlap_props_test['Unknown_Prop'].iloc[row] -= 200
    if overlap_props_test['Unknown_Prop'].iloc[row] >= 100.0:
        overlap_props_test['Unknown_Prop'].iloc[row] -= 100
overlap_props_test = round(overlap_props_test, 3)

# Overlap Proportions for Test group
overlap_props_test.mean()[1:]

# Information on Red and Yellow for Overlap Test
print('Proportion of individuals that took > 50% Red =', len(overlap_props_test[overlap_props_test['Red_Prop'] >= 50.0]) / len(overlap_props_test))
print('Proportion of individuals that took > 50% Red or Yellow =', len(overlap_props_test[(overlap_props_test['Red_Prop'] + overlap_props_test['Yellow_Prop']) >= 0.50]) / len(overlap_props_test))
print('Proportion of individuals that took 100% Red or Yellow =', len(overlap_props_test[(overlap_props_test['Red_Prop'] + overlap_props_test['Yellow_Prop']) == 100]) / len(overlap_props_test))

Proportion of individuals that took > 50% Red = 0.10152284263959391
Proportion of individuals that took > 50% Red or Yellow = 0.7106598984771574
Proportion of individuals that took 100% Red or Yellow = 0.4517766497461929


### Information on Other (Metformin without Oncologics) Data

This section contains information regarding the individuals that filed a claim for Metformin but no oncologic treatment. Again, the section using the **Test** ratings is designated as such, as is the **Assoc** ratings section. 

In [15]:
# Assoc # Number of people that took Metformin and zero Oncologics
len(set(met_claims_ids)) - len(set(overlap_ids))

7247

In [16]:
# Test # Number of people that took Metformin and zero Oncologics
len(set(met_claims_ids)) - len(set(overlap_tested_ids))

7271

In [17]:
# Assoc # Just Metformin takers taking Red/Yellow/Green Associated doses of Metformin
met_nocancer_assoc_claims_ids = list(set(met_claims_ids)-set(overlap_ids))
met_nocancer_assoc = slc_claims[slc_claims['MEMBER_KEY'].isin(met_nocancer_assoc_claims_ids)]
met_nocancer_assoc = pd.merge(met_nocancer_assoc, valisure_met_ratings, left_on = 'RX_NDC_CODE_9', right_on = 'NDC', how = 'left')
met_nocancer_assoc_ratings = met_nocancer_assoc[met_nocancer_assoc['Rating_Assoc'].notna()].groupby(['MEMBER_KEY', 'Rating_Assoc'])[['MEMBER_KEY', 'Rating_Assoc']].size().reset_index()
met_nocancer_assoc_ratings = met_nocancer_assoc_ratings.rename(columns = { 0:'#_Prescriptions' })
met_nocancer_assoc_ratings['Tainted'] = 'NA'
for member in range(len(met_nocancer_assoc_claims_ids)):
    ind = list(met_nocancer_assoc_ratings[met_nocancer_assoc_ratings['MEMBER_KEY'] == met_nocancer_assoc_claims_ids[member]]['Rating_Assoc'].index)
    if (any(met_nocancer_assoc_ratings[met_nocancer_assoc_ratings['MEMBER_KEY'] == met_nocancer_assoc_claims_ids[member]]['Rating_Assoc'] == 'red')):
        met_nocancer_assoc_ratings['Tainted'].iloc[ind] = 2
    elif any(met_nocancer_assoc_ratings[met_nocancer_assoc_ratings['MEMBER_KEY'] == met_nocancer_assoc_claims_ids[member]]['Rating_Assoc'] == 'yellow') and all(met_nocancer_assoc_ratings[met_nocancer_assoc_ratings['MEMBER_KEY'] == met_nocancer_assoc_claims_ids[member]]['Rating_Assoc'] != 'red'):
        met_nocancer_assoc_ratings['Tainted'].iloc[ind] = 1
    else:
        met_nocancer_assoc_ratings['Tainted'].iloc[ind] = 0
met_nocancer_assoc_ratings = met_nocancer_assoc_ratings.iloc[met_nocancer_assoc_ratings.iloc[:, [0, 3]].drop_duplicates().index]
ids_tainted_assoc_red_nocancer = met_nocancer_assoc_ratings[met_nocancer_assoc_ratings['Tainted'] == 2].count()[0]
ids_tainted_assoc_yellow_nocancer = met_nocancer_assoc_ratings[met_nocancer_assoc_ratings['Tainted'] == 1].count()[0]
ids_clean_assoc_nocancer = len(met_nocancer_assoc_claims_ids) - ids_tainted_assoc_red_nocancer - ids_tainted_assoc_yellow_nocancer
print((ids_tainted_assoc_red_nocancer + ids_tainted_assoc_yellow_nocancer), 'out of', len(met_nocancer_assoc_claims_ids), 'IDs took tainted Metformin')
print(ids_tainted_assoc_red_nocancer, 'IDs took Red rated Metformin')
print(ids_tainted_assoc_yellow_nocancer, 'IDs took Yellow rated Metformin')
print(ids_clean_assoc_nocancer, 'IDs were clean/unknown')
print('Red rate =', round(ids_tainted_assoc_red_nocancer/len(met_nocancer_assoc_claims_ids), 3))
print('Yellow rate =', round(ids_tainted_assoc_yellow_nocancer/len(met_nocancer_assoc_claims_ids), 3))
print('Tainted rate =', round((ids_tainted_assoc_red_nocancer + ids_tainted_assoc_yellow_nocancer)/len(met_nocancer_assoc_claims_ids), 3))

5033 out of 7247 IDs took tainted Metformin
1248 IDs took Red rated Metformin
3785 IDs took Yellow rated Metformin
2214 IDs were clean/unknown
Red rate = 0.172
Yellow rate = 0.522
Tainted rate = 0.694


In [18]:
# Test # Just Metformin takers taking Red/Yellow/Green Tested doses of Metformin
# Tainted (test) Metformin rates for non-cancer takers
met_nocancer_test_claims_ids = list(set(met_claims_ids)-set(overlap_tested_ids))
met_nocancer_test = slc_claims[slc_claims['MEMBER_KEY'].isin(met_nocancer_test_claims_ids)]
met_nocancer_test = pd.merge(met_nocancer_test, valisure_met_ratings, left_on = 'RX_NDC_CODE_9', right_on = 'NDC', how = 'left')
met_nocancer_test_ratings = met_nocancer_test[met_nocancer_test['Rating_Test'].notna()].groupby(['MEMBER_KEY', 'Rating_Test'])[['MEMBER_KEY', 'Rating_Test']].size().reset_index()
met_nocancer_test_ratings = met_nocancer_test_ratings.rename(columns = { 0:'#_Prescriptions' })
met_nocancer_test_ratings['Tainted'] = 'NA'
for member in range(len(met_nocancer_test_claims_ids)):
    ind = list(met_nocancer_test_ratings[met_nocancer_test_ratings['MEMBER_KEY'] == met_nocancer_test_claims_ids[member]]['Rating_Test'].index)
    if (any(met_nocancer_test_ratings[met_nocancer_test_ratings['MEMBER_KEY'] == met_nocancer_test_claims_ids[member]]['Rating_Test'] == 'red')):
        met_nocancer_test_ratings['Tainted'].iloc[ind] = 2
    elif (any(met_nocancer_test_ratings[met_nocancer_test_ratings['MEMBER_KEY'] == met_nocancer_test_claims_ids[member]]['Rating_Test'] == 'yellow') and all(met_nocancer_test_ratings[met_nocancer_test_ratings['MEMBER_KEY'] == met_nocancer_test_claims_ids[member]]['Rating_Test'] != 'red')):
        met_nocancer_test_ratings['Tainted'].iloc[ind] = 1
    else:
        met_nocancer_test_ratings['Tainted'].iloc[ind] = 0
met_nocancer_test_ratings = met_nocancer_test_ratings.iloc[met_nocancer_test_ratings.iloc[:, [0, 3]].drop_duplicates().index]
ids_tainted_test_red_nocancer = met_nocancer_test_ratings[met_nocancer_test_ratings['Tainted'] == 2].count()[0]
ids_tainted_test_yellow_nocancer = met_nocancer_test_ratings[met_nocancer_test_ratings['Tainted'] == 1].count()[0]
ids_clean_test_nocancer = len(met_nocancer_test_claims_ids) - ids_tainted_test_red_nocancer - ids_tainted_test_yellow_nocancer
print((ids_tainted_test_red_nocancer + ids_tainted_test_yellow_nocancer), 'out of', len(met_nocancer_test_claims_ids), 'IDs took tainted Metformin')
print(ids_tainted_test_red_nocancer, 'IDs took Red rated Metformin')
print(ids_tainted_test_yellow_nocancer, 'IDs took Yellow rated Metformin')
print(ids_clean_test_nocancer, 'IDs were clean/unknown')
print('Red rate =', round(ids_tainted_test_red_nocancer/len(met_nocancer_test_claims_ids), 3))
print('Yellow rate =', round(ids_tainted_test_yellow_nocancer/len(met_nocancer_test_claims_ids), 3))
print('Tainted rate =', round((ids_tainted_test_red_nocancer + ids_tainted_test_yellow_nocancer)/len(met_nocancer_test_claims_ids), 3))

4725 out of 7271 IDs took tainted Metformin
980 IDs took Red rated Metformin
3745 IDs took Yellow rated Metformin
2546 IDs were clean/unknown
Red rate = 0.135
Yellow rate = 0.515
Tainted rate = 0.65


Below is information regarding the distribution of Metformin that Metformin-only individuals in different groups took based upon Valisure ratings.

In [19]:
# Create Metformin table and 'TOTAL_UNITS' for Metformin
met_claims = pd.merge(met_claims, valisure_met_ratings, left_on = 'RX_NDC_CODE_9', right_on = 'NDC', how = 'left').drop('NDC', axis = 1)
for row in range(len(met_claims)):
    if pd.isnull(met_claims['Rating_Assoc'][row]):
        met_claims['Rating_Assoc'][row] = 'unknown'
    if pd.isnull(met_claims['Rating_Test'][row]):
        met_claims['Rating_Test'][row] = 'unknown'
met_claims_cond = met_claims.iloc[:, [0, 1, 2, 5, 6, 8, 9, 10, 11, 12, 21, 22, 23]]
met_claims_cond['TOTAL_UNITS'] = met_claims['QTY_SVC_UNITS'] * met_claims['RX_REFILLS']

In [20]:
# Assoc # Create 'TOTAL_UNITS' for proportions for Metformin (Assoc)
met_claims_cond_assoc = met_claims_cond.loc[met_claims_cond['MEMBER_KEY'].isin(met_nocancer_assoc_claims_ids)]
met_claims_assoc = met_claims_cond_assoc.groupby(['MEMBER_KEY', 'Rating_Assoc'])['TOTAL_UNITS'].sum()
met_claims_assoc_ind = met_claims_cond.groupby(['MEMBER_KEY'])['TOTAL_UNITS'].sum()
met_claims_assoc = met_claims_assoc.div(met_claims_assoc_ind, level = 'MEMBER_KEY').reset_index()
met_claims_assoc['TOTAL_UNITS'] = met_claims_assoc['TOTAL_UNITS'] * 100

# Create Table for proportion of each quality (Assoc)
met_claims_assoc['Red_Prop'] = 0.0
met_claims_assoc['Yellow_Prop'] = 0.0
met_claims_assoc['Green_Prop'] = 0.0
met_claims_assoc['Unknown_Prop'] = 0.0
for row in range(len(met_claims_assoc)):
    if met_claims_assoc['Rating_Assoc'][row] == 'red':
        met_claims_assoc['Red_Prop'][row] = met_claims_assoc['TOTAL_UNITS'][row]
    elif met_claims_assoc['Rating_Assoc'][row] == 'yellow':
        met_claims_assoc['Yellow_Prop'][row] = met_claims_assoc['TOTAL_UNITS'][row]
    elif met_claims_assoc['Rating_Assoc'][row] == 'green':
        met_claims_assoc['Green_Prop'][row] = met_claims_assoc['TOTAL_UNITS'][row]
    met_claims_assoc['Unknown_Prop'][row] = 100 - (met_claims_assoc['Red_Prop'][row] + met_claims_assoc['Yellow_Prop'][row] + met_claims_assoc['Green_Prop'][row])
met_claims_assoc = met_claims_assoc.drop(['Rating_Assoc'], axis = 1).groupby('MEMBER_KEY').sum()

# Fix proportions > 100% (Assoc)
for row in range(len(overlap_props_assoc)):
    if met_claims_assoc['Unknown_Prop'].iloc[row] >= 200.0:
        met_claims_assoc['Unknown_Prop'].iloc[row] -= 200
    if met_claims_assoc['Unknown_Prop'].iloc[row] >= 100.0:
        met_claims_assoc['Unknown_Prop'].iloc[row] -= 100
met_claims_assoc = round(met_claims_assoc, 3)

# Overlap Proportions for Assoc group
met_claims_assoc.mean()[1:]

# Information on Red and Yellow for Overlap Assoc
print('Proportion of individuals that took > 50% Red =', len(met_claims_assoc[met_claims_assoc['Red_Prop'] >= 50.0]) / len(met_claims_assoc))
print('Proportion of individuals that took > 50% Red or Yellow =', len(met_claims_assoc[(met_claims_assoc['Red_Prop'] + met_claims_assoc['Yellow_Prop']) >= 0.50]) / len(met_claims_assoc))
print('Proportion of individuals that took 100% Red or Yellow =', len(met_claims_assoc[(met_claims_assoc['Red_Prop'] + met_claims_assoc['Yellow_Prop']) == 100]) / len(met_claims_assoc))

Proportion of individuals that took > 50% Red = 0.08265489167931558
Proportion of individuals that took > 50% Red or Yellow = 0.6768317924658479
Proportion of individuals that took 100% Red or Yellow = 0.43838829860631984


In [21]:
# Test # Create 'TOTAL_UNITS' for proportions for Metformin
met_claims_cond_test = met_claims_cond.loc[met_claims_cond['MEMBER_KEY'].isin(met_nocancer_test_claims_ids)]
met_claims_test = met_claims_cond.groupby(['MEMBER_KEY', 'Rating_Test'])['TOTAL_UNITS'].sum()
met_claims_test_ind = met_claims_cond.groupby(['MEMBER_KEY'])['TOTAL_UNITS'].sum()
met_claims_test = met_claims_test.div(met_claims_test_ind, level = 'MEMBER_KEY').reset_index()
met_claims_test['TOTAL_UNITS'] = met_claims_test['TOTAL_UNITS'] * 100

# Create Table for proportion of each quality (Test)
met_claims_test['Red_Prop'] = 0.0
met_claims_test['Yellow_Prop'] = 0.0
met_claims_test['Green_Prop'] = 0.0
met_claims_test['Unknown_Prop'] = 0.0
for row in range(len(met_claims_test)):
    if met_claims_test['Rating_Test'][row] == 'red':
        met_claims_test['Red_Prop'][row] = met_claims_test['TOTAL_UNITS'][row]
    elif met_claims_test['Rating_Test'][row] == 'yellow':
        met_claims_test['Yellow_Prop'][row] = met_claims_test['TOTAL_UNITS'][row]
    elif met_claims_test['Rating_Test'][row] == 'green':
        met_claims_test['Green_Prop'][row] = met_claims_test['TOTAL_UNITS'][row]
    met_claims_test['Unknown_Prop'][row] = 100 - (met_claims_test['Red_Prop'][row] + met_claims_test['Yellow_Prop'][row] + met_claims_test['Green_Prop'][row])
met_claims_test = met_claims_test.drop(['Rating_Test'], axis = 1).groupby('MEMBER_KEY').sum()

# Fix proportions > 100% (Test)
for row in range(len(met_claims_test)):
    if met_claims_test['Unknown_Prop'].iloc[row] >= 200.0:
        met_claims_test['Unknown_Prop'].iloc[row] -= 200
    if met_claims_test['Unknown_Prop'].iloc[row] >= 100.0:
        met_claims_test['Unknown_Prop'].iloc[row] -= 100
met_claims_test = round(met_claims_test, 3)

# Overlap Proportions for Test group
met_claims_test.mean()[1:]

# Information on Red and Yellow for Overlap Test
print('Proportion of individuals that took > 50% Red =', len(met_claims_test[met_claims_test['Red_Prop'] >= 50.0]) / len(met_claims_test))
print('Proportion of individuals that took > 50% Red or Yellow =', len(met_claims_test[(met_claims_test['Red_Prop'] + met_claims_test['Yellow_Prop']) >= 0.50]) / len(met_claims_test))
print('Proportion of individuals that took 100% Red or Yellow =', len(met_claims_test[(met_claims_test['Red_Prop'] + met_claims_test['Yellow_Prop']) == 100]) / len(met_claims_test))

Proportion of individuals that took > 50% Red = 0.07204070701660417
Proportion of individuals that took > 50% Red or Yellow = 0.6355115158007498
Proportion of individuals that took 100% Red or Yellow = 0.4054633101231923


### Medication Adherence between groups

The number of days of Metformin taken across the year for each individual is worth noting, and if there is a vast difference between the overlap group and no-cancer group, this would be important in an eventual study.

In [22]:
# Overlap/Cancer # Determine patient adherence to Metformin
overlap_cond['Days_Adherence'] = overlap_cond['QTY_RX_DAYS_SUPPLY'] * overlap_cond['RX_REFILLS']
overlap_adherence = overlap_cond.groupby(['MEMBER_KEY', 'FROM_DATE_YEAR'])['Days_Adherence'].sum().reset_index()
overlap_adherence['Days_Adherence'] = list(np.where(overlap_adherence['Days_Adherence'] > 365, 365, overlap_adherence['Days_Adherence']))
print('Average annual days adherence:', round(np.mean(overlap_adherence['Days_Adherence']), ))

Average annual days adherence: 286.0


In [23]:
# No-Cancer # Determine patient adherence to Metformin
met_claims_cond['Days_Adherence'] = met_claims_cond['QTY_RX_DAYS_SUPPLY'] * met_claims_cond['RX_REFILLS']
met_claims_adherence = met_claims_cond.groupby(['MEMBER_KEY', 'FROM_DATE_YEAR'])['Days_Adherence'].sum().reset_index()
met_claims_adherence['Days_Adherence'] = list(np.where(met_claims_adherence['Days_Adherence'] > 365, 365, met_claims_adherence['Days_Adherence']))
print('Average annual days adherence:', round(np.mean(met_claims_adherence['Days_Adherence']), 2))

Average annual days adherence: 278.54


### T-Tests between groups

As a primary look at the different incidence of red and tainted (red + yellow) Metformin as a proportion of the total Metformin intake between the cancer and nocancer groups, I examined some simple Student's t-Tests between groups. 

In [24]:
# T-Tests for amounts of Red taken between overlap and nocancer
overlap_test_red = list(overlap_props_test['Red_Prop'])
overlap_assoc_red = list(overlap_props_assoc['Red_Prop'])
met_claims_test_red = list(met_claims_test['Red_Prop'])
met_claims_assoc_red = list(met_claims_assoc['Red_Prop'])
t_test, p_test = stats.ttest_ind(overlap_test_red, met_claims_test_red)
t_assoc, p_assoc = stats.ttest_ind(overlap_assoc_red, met_claims_assoc_red)
print('P-value (test) =', p_test, '-- P-value (assoc) =', p_assoc)

P-value (test) = 0.052701466196960124 -- P-value (assoc) = 0.11428610087925622


In [25]:
# T-Tests for amounts of Tainted (Red + Yellow) taken
overlap_test_tainted = list(overlap_props_test['Red_Prop'] + overlap_props_test['Yellow_Prop'])
overlap_assoc_tainted = list(overlap_props_assoc['Red_Prop'] + overlap_props_assoc['Yellow_Prop'])
met_claims_test_tainted = list(met_claims_test['Red_Prop'] + met_claims_test['Yellow_Prop'])
met_claims_assoc_tainted = list(met_claims_assoc['Red_Prop'] + met_claims_assoc['Yellow_Prop'])
t_test2, p_test2 = stats.ttest_ind(overlap_test_tainted, met_claims_test_tainted)
t_assoc2, p_assoc2 = stats.ttest_ind(overlap_assoc_tainted, met_claims_assoc_tainted)
print('P-value (test) =', p_test2, '-- P-value (assoc) =', p_assoc2)

P-value (test) = 0.02856631922035092 -- P-value (assoc) = 0.09915047902997741


### Other Informative Statistics

In [26]:
# Drugs without linked NDC
no_ndc = len(oncologics_ndcs['drugname'].unique()) - len(oncologics_ndcs_full['drugname'].unique())
print(no_ndc, 'of', len(oncologics_ndcs['drugname'].unique()), 'drugs without associated NDC')
print()

# Number of Metformin NDCs not evaluated by Valisure
print(len(met_product_ndcs - set([sl[:9] for sl in list(set(valisure_met_ndcs))])), 'Metformin NDCs not evaluated by Valisure')
print()

# Size of Overlap & No-Cancer Groups
print('Group Sizes')
print('Overlap: for Assoc,', len(overlap_ids), '-- for Test,', len(overlap_tested_ids))
print('No-Cancer: for Assoc,', len(set(met_claims_ids)) - len(set(overlap_ids)), '-- for Test,', len(set(met_claims_ids)) - len(set(overlap_tested_ids)))
print()

# Average number of claims for individuals in the Overlap
print(round(len(overlap_all) / overlap_all['MEMBER_KEY'].nunique(), 3), 'claims per individual in Overlap group')

# Average number of claims for individuals in the No-Cancer group
print(round(len(slc_claims[~slc_claims['MEMBER_KEY'].isin(overlap_ids)]) / (slc_claims['MEMBER_KEY'].nunique() - len(overlap_ids)), 3), 'claims per individual in No-Cancer group')
print()

# Average number of different Metformin NDCs taken by individuals in Overlap group
overlap_all_diffndcs = overlap_all[overlap_all['RX_NDC_CODE'].isin(met_package_ndcs)].groupby('MEMBER_KEY')['RX_NDC_CODE'].nunique().reset_index().groupby('RX_NDC_CODE').count().reset_index()
overlap_all_diffndcs.columns = ['# of Diff NDCs', '# of Individuals']
overlap_all_diffndcs

538 of 714 drugs without associated NDC

57 Metformin NDCs not evaluated by Valisure

Group Sizes
Overlap: for Assoc, 221 -- for Test, 197
No-Cancer: for Assoc, 7247 -- for Test, 7271

70.498 claims per individual in Overlap group
20.143 claims per individual in No-Cancer group



,# of Diff NDCs,# of Individuals
0,1,127
1,2,56
2,3,20
3,4,4
4,5,1
5,7,1
